In [1]:
import pandas as pd

data_raw = pd.read_csv('/root/xuying_experiments/stock_data.csv')
data_raw.head(3)

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1


In [2]:
import torch
from sentence_transformers import SentenceTransformer

# set device to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# load the retriever model from huggingface model hub
retriever = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1', device=device)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [62]:
from tqdm.auto import tqdm

# we will use batches of 1
batch_size = 1
res = data_raw['Text'].values.tolist()
res_feature = []

for i in tqdm(range(0, len(res), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(res))
    
    # extract batch
    batch = res[i:i_end]
    # generate embeddings for batch
    emb = retriever.encode(batch).tolist()[0]
    # get metadata
    # meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    emb = torch.FloatTensor(emb)
    res_feature.append(emb)

  0%|          | 0/5791 [00:00<?, ?it/s]

In [63]:
len(res_feature[0])

384

In [8]:
import sklearn as sk
from sklearn.linear_model import LogisticRegression

In [67]:
data_raw['feature'] = res_feature

In [73]:
labels = []
for i in data_raw['Sentiment']:
    labels.append(torch.tensor(i))

In [69]:
import numpy as np
import torch
train_features = torch.cat(res_feature).cpu().numpy()

In [84]:
labels = np.array(data_raw['Sentiment'])

In [56]:
labels = labels.reshape(-1, 1)

In [78]:
len(train_features)

2223744

In [102]:
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(torch.stack(res_feature), data_raw['Sentiment'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


LogisticRegression(C=0.316, max_iter=1000, random_state=0, verbose=1)

In [103]:
# Evaluate using the logistic regression classifier
predictions = classifier.predict(torch.stack(res_feature))
accuracy = np.mean((data_raw['Sentiment'] == predictions).astype(float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Accuracy = 73.994
